# ***Coleta final dos dados***
---

> COM DADOS DE CATEGORIAS DO GOOGLE TRENDS

## Descrição

A partir dos scripts de teste individuais, criaremos funções para coletar todos os dados e, posteriormente, juntarmos todos os dados em um dataframe final: `now_data_raw.csv` e `.xlsx`.

# Bibliotecas
---

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from functools import reduce

import sys
sys.path.insert(0, '..')

from bcb import sgs
import ipeadatapy as ip
import pandas_datareader.data as web

import yfinance as yfin


yfin.pdr_override()

# Definindo funções

In [2]:
def get_bcb_data(path, start, end):
    # Lendo a planilha completa
    df1 = pd.read_csv(path)

    # Filtrando apenas para os dados que constam no SGS-BCB
    df2 = df1[df1['where'] == 'BCB']

    # Definindo os nomes e os códigos das séries
    names_df = list(df2['name_df'])
    codes_bcb = list(map(int, df2['code']))

    # Coletando as séries com o API: criando uma lista compacta com os nomes e códigos para coletar todos os dados simultâneamente
    df3 = sgs.get(list(zip(names_df, codes_bcb)), start = start, end = end, multi = True) # multi: coletando diversas séries em uma única requisição
    df3 = df3.reset_index()

    return df3

In [3]:
def get_ipea_data(path, year_greater):
    # Lendo a planilha completa
    df1 = pd.read_csv(path)

    # Filtrando apenas para os dados que constam no Ipeadata
    df2 = df1[df1['where'] == 'Ipeadata']

    # Criando lista de códigos
    codes = list(df2['code'])

    # Coletando os dados
    df3 = pd.DataFrame()
    for code in codes:
        df3[code] = ip.timeseries(series = code, yearGreaterThan = year_greater).iloc[:, -1:]
    
    df3 = df3.reset_index()

    return df3

In [4]:
def get_data_yahoo(path, start, end, interval, series):
    # Lendo a planilha completa
    df1 = pd.read_csv(path)

    # Filtrando apenas para os dados que constam no Yahoo! Finance
    df2 = df1[df1['where'] == 'Yahoo! Finance']

    # Criando lista de códigos
    codes = list(df2['code'])

    # Coletando os dados
    df3 = pd.DataFrame()

    for code in codes:
        df3[code] = web.get_data_yahoo(code, start = start, end = end, interval = interval)[series]
    
    df3 = df3.reset_index()

    return df3

# Banco Central do Brasil (BCB)

In [5]:
'''path = 'meta-data-gtcat.csv'
start = '2002-01-01'
end = '2022-09-01'
bcb_raw = get_bcb_data(path, start, end)
bcb_raw'''

bcb_raw = pd.read_csv('../bcb_data_raw.csv')
bcb_raw.drop('Unnamed: 0', axis = 1, inplace = True)
bcb_raw.rename({'ref.date' : 'Date'}, axis = 1, inplace = True)
bcb_raw['Date'] = pd.to_datetime(bcb_raw['Date'])
bcb_raw['Date'] = bcb_raw['Date'].dt.strftime('%Y-%m-%d')

# BCB Focus - Expectativas de mercado 

In [6]:
focus_raw = pd.read_csv('focus_data.csv')
focus_raw.drop('Unnamed: 0', axis = 1, inplace = True)
focus_raw['Date'] = pd.to_datetime(bcb_raw['Date'])
focus_raw['Date'] = focus_raw['Date'].dt.strftime('%Y-%m-%d')

# Ipeadata

In [7]:
path = 'meta-data-gtcat.csv'
year_greater = 2001
ipea_raw = get_ipea_data(path, year_greater)
ipea_raw.columns = ['Date', 'exp_fob', 'exp_kgood_fob', 'exp_comgood_fob', 'imp_fob', 'imp_kgood_fob', 'imp_comgood_fob', 'revenue_rf']
ipea_raw = ipea_raw[ipea_raw['Date'] <= '2022-09-01']
ipea_raw['Date'] = ipea_raw['Date'].dt.strftime('%Y-%m-%d')

# Yahoo! Finance
---

In [8]:
path = 'meta_data.csv'
start = '2002-01-01'
end = '2022-10-01'
interval = '1mo'
series = 'Adj Close'
yahoo_raw = get_data_yahoo(path, start, end, interval, series)
# yahoo_raw.drop('^N100', axis = 1, inplace = True)
yahoo_raw.columns = ['Date', 'brent_oil_price', 'ibov', 'nasdaq', 'snp_500', 'dji']
yahoo_raw['Date'] = yahoo_raw['Date'].dt.strftime('%Y-%m-%d')
yahoo_raw

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,brent_oil_price,ibov,nasdaq,snp_500,dji
0,2007-08-01,72.690002,54637.0,2596.360107,1473.989990,13357.740234
1,2007-09-01,79.169998,60465.0,2701.500000,1526.750000,13895.629883
2,2007-10-01,90.629997,65318.0,2859.120117,1549.380005,13930.009766
3,2007-11-01,88.260002,63006.0,2660.959961,1481.140015,13371.719727
4,2007-12-01,93.849998,63886.0,2652.280029,1468.359985,13264.820312
...,...,...,...,...,...,...
151,2022-04-01,109.339996,107876.0,12334.639648,4131.930176,32977.210938
152,2022-06-01,114.809998,98542.0,11028.740234,3785.379883,30775.429688
153,2022-07-01,110.010002,103165.0,12390.690430,4130.290039,32845.128906
154,2022-08-01,96.489998,109523.0,11816.200195,3955.000000,31510.429688


# Juntando as bases de dados
---

In [9]:
# Lendo base de dados da FGV com os índices de confiança 

fgv_raw = pd.read_excel('fgv_expect_nuci.xlsx')
fgv_raw.columns = ['Date', 'emp_conf_index_fgv', 'emp_curr_situ_index_fgv', 'emp_expec_index_fgv', 'ind_conf_index_fgv', 
                   'ind_curr_situ_index_fgv', 'ind_expec_index_fgv', 'nuci_gv', 'cons_conf_index_fgv', 'cons_curr_situ_index_fgv', 
                   'cons_expec_index_fgv']
fgv_raw['Date'] = pd.date_range(start = '2002-01-01', end = '2022-09-01', freq = 'MS')
fgv_raw.set_index(['Date'], inplace = True)
fgv_raw = fgv_raw.reset_index()
fgv_raw['Date'] = fgv_raw['Date'].dt.strftime("%Y-%m-%d")
fgv_raw

,Date,emp_conf_index_fgv,emp_curr_situ_index_fgv,emp_expec_index_fgv,ind_conf_index_fgv,ind_curr_situ_index_fgv,ind_expec_index_fgv,nuci_gv,cons_conf_index_fgv,cons_curr_situ_index_fgv,cons_expec_index_fgv
0,2002-01-01,93.0,93.7,92.6,94.9,95.5,94.8,78.6,NaN,NaN,NaN
1,2002-02-01,97.1,93.8,100.5,98.9,95.6,102.5,79.2,NaN,NaN,NaN
2,2002-03-01,97.8,92.8,102.9,99.6,94.6,104.8,78.3,NaN,NaN,NaN
3,2002-04-01,99.3,93.0,105.6,101.1,94.8,107.5,77.4,NaN,NaN,NaN
4,2002-05-01,95.2,88.2,102.6,97.0,90.2,104.5,77.6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
244,2022-05-01,96.7,97.3,96.5,100.4,100.4,100.6,80.3,75.3,68.4,81.6
245,2022-06-01,96.8,97.3,96.5,101.4,101.1,101.5,80.8,78.1,68.7,86.0
246,2022-07-01,97.3,97.8,97.4,101.0,100.0,102.1,81.7,78.6,69.5,86.3
247,2022-08-01,100.2,99.5,100.9,103.0,102.2,103.5,82.4,82.9,71.3,92.1


In [10]:
# Lendo base de dados Google Trends

gtrends_cats = pd.read_csv('gt-cats-raw.csv')
gtrends_cats

,Date,gt_arts,gt_books,gt_finance,gt_health,gt_internet,gt_news,gt_pets,gt_science,gt_travel,...,gt_realestate,gt_shopping,gt_bfit,gt_pceletro,gt_games,gt_home,gt_law,gt_povo,gt_ref,gt_sports
0,2004-01-01,28.125,40.625,22.250,19.875,18.625,11.000,21.500,44.875,41.125,...,25.750,30.125,17.875,67.875,26.750,11.250,32.750,35.750,24.750,8.750
1,2004-02-01,27.625,43.875,21.500,20.500,16.125,10.750,21.625,61.500,33.500,...,21.375,27.875,16.125,69.250,23.625,11.875,33.375,35.000,28.375,8.125
2,2004-03-01,31.500,61.125,25.250,28.500,18.875,13.000,27.250,93.625,34.125,...,23.500,33.125,19.250,80.250,24.375,13.000,42.750,44.250,39.625,10.125
3,2004-04-01,31.750,60.000,25.625,29.000,19.125,12.125,25.750,85.875,34.250,...,21.500,32.375,20.125,77.000,24.375,13.000,42.500,45.750,38.375,11.125
4,2004-05-01,33.250,66.125,25.875,33.500,21.750,13.875,28.875,93.000,38.375,...,22.625,35.625,21.000,84.000,25.125,15.125,43.625,49.250,42.500,11.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2022-05-01,53.125,66.250,82.500,86.375,46.625,54.625,73.000,98.750,80.375,...,68.250,91.625,83.875,76.250,51.875,73.875,73.125,92.125,93.625,81.250
221,2022-06-01,53.625,64.250,73.750,92.250,47.250,56.375,75.750,93.125,82.750,...,68.000,90.875,89.500,79.375,54.625,75.750,69.875,91.875,94.500,86.000
222,2022-07-01,55.625,59.625,76.375,91.000,48.000,50.500,79.500,78.625,87.125,...,71.500,89.500,93.000,75.750,55.750,79.000,73.625,91.750,90.375,97.750
223,2022-08-01,54.875,71.125,78.250,90.875,50.000,61.500,78.125,96.125,81.625,...,76.125,89.500,95.000,80.125,54.875,77.875,78.750,95.500,98.375,98.125


In [11]:
# Lendo base de dados Google Trends -WORDS

gtrends_words = pd.read_csv('gt-words-raw.csv')
gtrends_words

,Date,gt_crise,gt_desemprego,gt_segdesemp,gt_empregos,gt_trabalho,gt_fgts,gt_renda,gt_salario,gt_economia,...,gt_politica,gt_aluguel,gt_seguro,gt_governo,gt_viagem,gt_empreendedor,gt_empresa,gt_credito,gt_dolar,gt_financiamento
0,2004-01-01,11.000,20.125,14.875,100.000,49.000,7.000,25.375,17.250,42.750,...,26.000,100.000,72.125,7.000,90.375,14.375,79.750,53.250,9.125,56.500
1,2004-02-01,23.250,19.875,12.625,83.625,67.625,5.375,34.000,18.125,55.750,...,42.250,73.750,53.500,5.875,69.625,25.375,59.625,54.625,7.125,45.625
2,2004-03-01,39.750,32.500,11.250,75.375,92.250,3.875,62.125,15.875,93.875,...,71.125,54.375,41.125,3.875,59.000,26.750,79.375,52.250,8.375,44.375
3,2004-04-01,34.625,38.625,14.125,69.625,99.500,4.000,90.000,24.500,83.500,...,79.125,49.750,40.500,4.000,67.500,30.000,61.375,48.625,8.000,38.375
4,2004-05-01,40.750,42.500,11.375,60.375,96.000,5.625,26.750,28.750,100.000,...,78.875,48.625,37.000,5.875,60.500,32.000,55.750,51.500,10.000,44.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2022-05-01,39.250,28.250,25.875,17.250,44.875,20.000,89.500,68.250,26.625,...,22.250,66.375,60.500,20.000,76.875,59.000,38.750,69.875,74.750,66.875
221,2022-06-01,39.875,29.250,25.375,16.250,41.375,12.625,52.750,62.625,25.750,...,21.000,65.625,62.625,12.750,79.750,52.625,39.125,62.625,65.375,64.250
222,2022-07-01,31.500,27.000,25.250,15.875,34.625,9.625,45.750,66.500,18.000,...,17.375,72.000,63.875,10.000,84.625,47.000,34.625,67.000,61.875,63.375
223,2022-08-01,36.625,30.250,26.250,16.750,42.750,9.875,41.750,67.500,27.250,...,26.625,68.000,63.750,10.000,78.750,47.875,37.875,63.375,59.125,67.875


In [12]:
dfs = [bcb_raw, fgv_raw, focus_raw, ipea_raw, yahoo_raw, gtrends_cats, gtrends_words]

now_data_raw = reduce(lambda left, right: pd.merge(left, right, on = ['Date'], how = 'outer'), dfs)

# Deslocando duas linhas abaixo para as séries trimestrais para que o lançamento dos dados fique no fim do período

now_data_raw['gdp'] = now_data_raw['gdp'].shift(2)

# Escrevendo em csv e xlsx

# now_data_raw.to_excel('now_data_raw.xlsx')
now_data_raw.to_csv('now_data_raw_full.csv')

In [13]:
now_data_raw

,Date,gdp,ibc_br,mon_gdp,ind_prod,ind_prod_ext,ind_prod_transf,ind_cap_goods,ind_prod_int_goods,ind_prod_man,...,gt_politica,gt_aluguel,gt_seguro,gt_governo,gt_viagem,gt_empreendedor,gt_empresa,gt_credito,gt_dolar,gt_financiamento
0,2002-01-01,NaN,NaN,112374.8,84.4,72.3,85.1,58.2,88.0,86.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002-02-01,NaN,NaN,111477.1,81.2,68.3,82.1,56.8,85.2,82.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002-03-01,110.63,NaN,118444.7,90.2,78.5,91.0,64.7,94.9,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002-04-01,NaN,NaN,120385.9,92.8,77.1,93.8,68.9,96.0,94.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002-05-01,NaN,NaN,123552.5,93.7,79.0,94.7,66.7,99.1,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2022-05-01,NaN,142.76,826040.2,102.7,95.2,103.9,108.4,101.5,103.9,...,22.250,66.375,60.500,20.00,76.875,59.000,38.750,69.875,74.750,66.875
245,2022-06-01,177.74,142.35,826471.8,101.8,98.4,102.4,101.9,101.3,102.7,...,21.000,65.625,62.625,12.75,79.750,52.625,39.125,62.625,65.375,64.250
246,2022-07-01,NaN,149.51,853882.5,107.1,105.7,107.4,101.9,106.8,109.0,...,17.375,72.000,63.875,10.00,84.625,47.000,34.625,67.000,61.875,63.375
247,2022-08-01,NaN,149.60,853568.6,110.7,106.8,111.4,112.4,109.0,114.1,...,26.625,68.000,63.750,10.00,78.750,47.875,37.875,63.375,59.125,67.875


In [14]:
now_data_raw.columns

Index(['Date', 'gdp', 'ibc_br', 'mon_gdp', 'ind_prod', 'ind_prod_ext',
       'ind_prod_transf', 'ind_cap_goods', 'ind_prod_int_goods',
       'ind_prod_man',
       ...
       'gt_politica', 'gt_aluguel', 'gt_seguro', 'gt_governo', 'gt_viagem',
       'gt_empreendedor', 'gt_empresa', 'gt_credito', 'gt_dolar',
       'gt_financiamento'],
      dtype='object', length=196)

# Observações

1. Completar os NAs da série do salério mínimo com 1212,00 reais pois a base tem apenas esses NAs quando o valor do SM em voga era esse. Provável erro de cômputo na base de dados.

2. Descartar série de horas trabalhadas na produção (hrs_wkd_prod) da fiesp: série incompleta. 

3. Descartar série de Utilização da capacidade instalada - Geral (CNI) (uci_cni): série incompleta.

4. Descartar série de vendas de máquinas agrícolas (agri_mach_sales): série incompleta

5. Para o fluxo de veículos (road_flow_gross_vehi), vamos precisar de apenas uma decomposição para estacionarizar a série, dado que sua série já existe apenas na forma dessazonalizada, poupando uma defasagem.

6. Decobrir coleta de PMIs